# Project 4: Web Scraping Indeed.com & Predicting Salaries

In Project 4, we practice two major skills: collecting data via  web scraping and building a binary predictor with Logistic Regression.

We will collect salary information on data science jobs in a variety of markets. Using location, title, and job summary, we'll predict the salary of the job. For job posting sites, this is extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), extrapolating expected salary can help guide negotiations.

Normally, we can use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Q: Why would we want this to be a classification problem?
- A: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Section one focuses on scraping Indeed.com; then we use listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

Scrape job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries. First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract.

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [3]:
URL = "http://www.indeed.com/jobs?q=data+scientist&l=Atlanta%2C+GA"

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup

In [5]:
# read site in soup
r = requests.get(URL)
soup = BeautifulSoup(r.content, "lxml")

# Append to the full set of results
results = soup.findAll('div', { "class" : "result" })
#results
#results[1]

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some of the more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```

- Make sure these functions are robust and can handle cases where the data/field may not be available
- Test the functions on the results above

In [6]:
# get text
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        
# company
def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

# location
def get_location_from_result(result):
    return extract_text(result.find('span', {'class' : 'location'}))



# summary
def get_summary_from_result(result):
    return extract_text(result.find('span', {'class' : 'summary'}))



# title
def get_title_from_result(result):
    return extract_text(result.find('a', {'data-tn-element' : 'jobTitle'}))



# get salary if exists
def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()   
    return None

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results: the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try different city). The second controls where in the results to start and gives 10 results (so we can keep incrementing this by 10 to move further within the list).

#### Complete the following code to collect results from multiple cities and start points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [7]:
# specify city
YOUR_CITY = ['Atlanta','Newark','Dallas','New+York','Houston', 'Boston','Connecticut',"San+Francisco",'Seattle','Cleveland','Tampa','Miami']
import pandas as pd

In [8]:
#create template URL and max number of results (pages) to pull
# url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
# df =pd.DataFrame()
# # for loop to pull data with bs4
# for city in YOUR_CITY :
#     for start in range(1,500,10):
#         r = requests.get(url_template.format(city,start))
#         # Grab the results from the request (as above)
#         soup = BeautifulSoup(r.content)
#         # Append to the full set of results
#         results += soup.findAll('div', { "class" : "result" })
# #a = get_location_from_result(results) 
# for element in  range(len(results)):
#     company = get_company_from_result(results[element])
#     location =get_location_from_result(results[element])
#     summary=get_summary_from_result(results[element])
#     title=get_title_from_result(results[element])
#     salary=get_salary_from_result(results[element])
# #     df = df.append({"Location":location,"Summary":summary,"Title":title,"Salary":salary},ignore_index = True)
#     df = df.append({"Company":company,"Location":location,"Title":title,"Salary":salary,"Summary":summary},ignore_index = True)
# df  

In [10]:
df

NameError: name 'df' is not defined

In [11]:
df.shape

NameError: name 'df' is not defined

In [12]:
df1 = df.dropna()

NameError: name 'df' is not defined

In [13]:
df1.shape

NameError: name 'df1' is not defined

In [14]:
df1.head(4) 


NameError: name 'df1' is not defined

In [15]:
len(results)

15

In [16]:
df1.dtypes


NameError: name 'df1' is not defined

In [17]:
df2 = df1.drop_duplicates()


NameError: name 'df1' is not defined

In [18]:
df2.shape

NameError: name 'df2' is not defined

In [19]:
df2

NameError: name 'df2' is not defined

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [20]:
# combine data into dictionaries
# rows = []
# for result in results:
#     if result:
#         row = {'location': }

# # create dataframe
# import pandas as pd


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [21]:
# Filter to only the rows that have salary entries

# Remove duplicates

# Filter out salary entries referring to week, hour or month
df2 = df2[~(df2.Salary.astype('str').str.contains('hour'))]
df2 = df2[~(df2.Salary.astype('str').str.contains('month'))]
df2 = df2[~(df2.Salary.astype('str').str.contains('week'))]

df2.shape

NameError: name 'df2' is not defined

In [22]:
df2

NameError: name 'df2' is not defined

 Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [23]:
import re
import numpy as np
def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'   
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])
    return matches

In [24]:
l1 = []
for i in df2["Salary"]:
    a = extract_salary_average(i)
    l1.append(a)
    
len(l1 ) 
df2["Salary"] = l1

NameError: name 'df2' is not defined

In [25]:
df2.head(2)

NameError: name 'df2' is not defined

In [26]:
df2.shape


NameError: name 'df2' is not defined

In [27]:
df2["Salary"] = df2["Salary"].astype(float)

NameError: name 'df2' is not defined

In [28]:
median_Salary = df2["Salary"].median()

NameError: name 'df2' is not defined

In [29]:
type(median_Salary)

NameError: name 'median_Salary' is not defined

In [30]:
df2.dtypes

NameError: name 'df2' is not defined

In [31]:
df2["Salary_class"] = df2["Salary"].map(lambda x: 1 if x>=median_Salary else 0)

NameError: name 'df2' is not defined

In [32]:
df2

NameError: name 'df2' is not defined

In [33]:
# use '.map' to transform salary to new feature


In [34]:
# save scraped results as a CSV for Tableau/external viz
df2.to_csv('indeed salary_file',sep='\t', encoding='utf-8')

NameError: name 'df2' is not defined

## Predicting salaries using Logistic Regression

In [36]:
# load in the the data of scraped salaries
salary_data = pd.read_csv("indeed salary_file")

CParserError: Error tokenizing data. C error: Expected 4 fields in line 10, saw 6


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [ ]:
# calculate median and create feature with 1 as high salary


### Q: What is the baseline accuracy for this model?

It is 50% if we guess randomly, half the salaries will be below the median and half will be above.

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [ ]:
# create statsmodel and summary
import statsmodels.formula.api as sm


#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' or 'Manager' is in the title 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [ ]:
# create senior, director, and manager dummies
salary_data['is_senior'] = salary_data['title'].str.contains('Senior').astype(int) # example


#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [ ]:
# scale, (patsy optional), and fit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from patsy import dmatrix

scaler = StandardScaler()
model = LogisticRegression(penalty = 'l2', C=0.1)


#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [ ]:
from sklearn.cross_validation import cross_val_score

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']: # example
    

### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
model = LogisticRegression(penalty = 'l1', C=1.0)

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    

In [ ]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : X.design_info.column_names, 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)
df

#### Optional: Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients. Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary. Which entries have the highest predicted salaries?

# Bonus Section: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are most valuable? 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform()

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

In [ ]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : vectorizer.get_feature_names(), 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# retest L1 and L2 regularization
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV()


Score: | /24
------|-------
Identify: Problem Statement and Hypothesis | 
Acquire: Import Data using BeautifulSoup| 
Parse: Clean and Organize Data| 
Model: Perform Logistic Regression| 
Evaluate: Logistic Regression Results	|
Present: Blog Report with Findings and Recommendations		| 
Interactive Tableau visualizations | 
Regularization |
Bonus: Countvectorizer  | 